In [1]:
import pandas as pd
import numpy as np
import calendar
from time import time
from datetime import datetime
import re 
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import log_loss, accuracy_score, classification_report
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_train = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")
data_train_orig = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")

In [3]:
dept_list = sorted(list(data_train.DepartmentDescription.dropna().unique()))

In [4]:
weekdays = list(calendar.day_name)
dept_list_sum = dict.fromkeys(dept_list, np.sum)
weekday_dict = dict.fromkeys(weekdays, np.max)
feature_dict = {"TripType": np.max, 'NumItems': np.sum, 'Return': np.max}
feature_dict = {**feature_dict, **weekday_dict, **dept_list_sum}

In [5]:
def transform_data(data):
    dummies = pd.get_dummies(data.Weekday)
    data[dummies.columns] = dummies
    
    dummies = pd.get_dummies(data.DepartmentDescription)
    dummies = dummies.apply(lambda x: x*data["ScanCount"])
    data[dummies.columns] = dummies 

    data.loc[data.ScanCount < 0, 'Return'] = 1
    data.loc[data.Return != 1, 'Return'] = 0
    
    data = data.rename(columns={"ScanCount":"NumItems"})
    
    grouped = data.groupby("VisitNumber")
    grouped = grouped.aggregate(feature_dict)
    data = grouped[["TripType", "NumItems", "Return"] + weekdays + dept_list]

    return data

In [6]:
data_new = transform_data(data_train)

In [7]:
def add_category_counts(data):
    alist = []
    for array in np.asarray(data.loc[:, dept_list[0]:]):
        count = 0
        count = sum(x > 0 for x in array)
        alist.append(count)
    cat_counts = pd.DataFrame(alist)
    cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
    cat_counts = cat_counts.set_index(data.index)
    data.insert(3, 'CategoryCounts', cat_counts)
    return data

In [8]:
data_new_cat = add_category_counts(data_new)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
X = data_new_cat.drop('TripType', axis=1)

trip_types = sorted(data_new_cat.TripType.unique())
trip_types_map = dict(zip(trip_types, np.arange(0, len(trip_types))))
y = data_new_cat.TripType.map(trip_types_map)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
np.random.seed(42)

In [12]:
def create_model_3hl(nodes_l1, nodes_l2, nodes_l3, dropout_l1, dropout_l2, dropout_l3):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout_l1))
    model.add(Dense(output_dim=nodes_l2, activation="relu"))
    model.add(Dropout(dropout_l2))
    model.add(Dense(output_dim=nodes_l2, activation="relu"))
    model.add(Dropout(dropout_l3))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [13]:
model_3hl = KerasClassifier(build_fn=create_model_3hl, epochs=70, verbose=2)

In [14]:
nodes_l1 = np.arange(150, 200, 10)
nodes_l2 = np.arange(90, 150, 10)
nodes_l3 = np.arange(50, 90, 10)

dropout_l1 = np.arange(0.1, 0.4, 0.1)
dropout_l2 = np.arange(0.1, 0.4, 0.1)
dropout_l3 = np.arange(0.1, 0.4, 0.1)
batch_size = [500, 1000, 2000, 3000]

In [15]:
params3 = dict(nodes_l1=nodes_l1, nodes_l2=nodes_l2, nodes_l3=nodes_l3, 
               dropout_l1=dropout_l1, dropout_l2=dropout_l2, dropout_l3=dropout_l3, batch_size=batch_size)

In [16]:
params3

{'nodes_l1': array([150, 160, 170, 180, 190]),
 'nodes_l2': array([ 90, 100, 110, 120, 130, 140]),
 'nodes_l3': array([50, 60, 70, 80]),
 'dropout_l1': array([0.1, 0.2, 0.3, 0.4]),
 'dropout_l2': array([0.1, 0.2, 0.3, 0.4]),
 'dropout_l3': array([0.1, 0.2, 0.3, 0.4]),
 'batch_size': [500, 1000, 2000, 3000]}

In [17]:
scoring = ['neg_log_loss', 'accuracy']
cv = StratifiedKFold(n_splits=4)

In [18]:
RS3 = RandomizedSearchCV(estimator=model_3hl, param_distributions=params3, cv=cv, n_iter = 30, 
                         scoring=scoring, refit='accuracy', random_state=42, n_jobs=1, verbose=2)

In [19]:
RS3

RandomizedSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000029833AFA0B8>,
                   iid='warn', n_iter=30, n_jobs=1,
                   param_distributions={'batch_size': [500, 1000, 2000, 3000],
                                        'dropout_l1': array([0.1, 0.2, 0.3, 0.4]),
                                        'dropout_l2': array([0.1, 0.2, 0.3, 0.4]),
                                        'dropout_l3': array([0.1, 0.2, 0.3, 0.4]),
                                        'nodes_l1': array([150, 160, 170, 180, 190]),
                                        'nodes_l2': array([ 90, 100, 110, 120, 130, 140]),
                                        'nodes_l3': array([50, 60, 70, 80])},
                   pre_dispatch='2*n_jobs', random_state=42, refit='accuracy',
                   return_train_score=False,

In [20]:
start = datetime.now()
RS3.fit(X, y)
end = datetime.now()
str(end-start)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
W0905 01:53:35.492754 15156 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Fitting 4 folds for each of 30 candidates, totalling 120 fits
[CV] nodes_l3=70, nodes_l2=120, nodes_l1=150, dropout_l3=0.2, dropout_l2=0.2, dropout_l1=0.1, batch_size=3000 


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=150)`
  This is separate from the ipykernel package so we can avoid doing imports until
W0905 01:53:35.529729 15156 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 01:53:35.532727 15156 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0905 01:53:35.557711 15156 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0905 01:53:35.5687

Epoch 1/70
 - 3s - loss: 2.9129 - acc: 0.2682
Epoch 2/70
 - 2s - loss: 2.0276 - acc: 0.4660
Epoch 3/70
 - 2s - loss: 1.6533 - acc: 0.5562
Epoch 4/70
 - 2s - loss: 1.4672 - acc: 0.5953
Epoch 5/70
 - 2s - loss: 1.3614 - acc: 0.6144
Epoch 6/70
 - 2s - loss: 1.2831 - acc: 0.6302
Epoch 7/70
 - 2s - loss: 1.2350 - acc: 0.6359
Epoch 8/70
 - 2s - loss: 1.1923 - acc: 0.6435
Epoch 9/70
 - 2s - loss: 1.1642 - acc: 0.6502
Epoch 10/70
 - 2s - loss: 1.1381 - acc: 0.6530
Epoch 11/70
 - 2s - loss: 1.1224 - acc: 0.6558
Epoch 12/70
 - 2s - loss: 1.1040 - acc: 0.6578
Epoch 13/70
 - 2s - loss: 1.0921 - acc: 0.6602
Epoch 14/70
 - 2s - loss: 1.0791 - acc: 0.6598
Epoch 15/70
 - 2s - loss: 1.0711 - acc: 0.6640
Epoch 16/70
 - 2s - loss: 1.0561 - acc: 0.6669
Epoch 17/70
 - 2s - loss: 1.0420 - acc: 0.6691
Epoch 18/70
 - 2s - loss: 1.0352 - acc: 0.6700
Epoch 19/70
 - 2s - loss: 1.0235 - acc: 0.6727
Epoch 20/70
 - 2s - loss: 1.0228 - acc: 0.6730
Epoch 21/70
 - 2s - loss: 1.0126 - acc: 0.6754
Epoch 22/70
 - 2s - lo

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=150)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


...

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=170)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=90)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=90)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/70
 - 13s - loss: 2.8999 - acc: 0.2596
Epoch 2/70
 - 3s - loss: 1.9602 - acc: 0.4769
Epoch 3/70
 - 3s - loss: 1.6047 - acc: 0.5637
Epoch 4/70
 - 3s - loss: 1.4355 - acc: 0.6013
Epoch 5/70
 - 3s - loss: 1.3336 - acc: 0.6214
Epoch 6/70
 - 3s - loss: 1.2689 - acc: 0.6314
Epoch 7/70
 - 4s - loss: 1.2258 - acc: 0.6385
Epoch 8/70
 - 3s - loss: 1.1907 - acc: 0.6443
Epoch 9/70
 - 3s - loss: 1.1585 - acc: 0.6495
Epoch 10/70
 - 3s - loss: 1.1396 - acc: 0.6531
Epoch 11/70
 - 3s - loss: 1.1214 - acc: 0.6565
Epoch 12/70
 - 3s - loss: 1.1058 - acc: 0.6607
Epoch 13/70
 - 3s - loss: 1.0916 - acc: 0.6634
Epoch 14/70
 - 3s - loss: 1.0857 - acc: 0.6629
Epoch 15/70
 - 3s - loss: 1.0677 - acc: 0.6662
Epoch 16/70
 - 3s - loss: 1.0575 - acc: 0.6691
Epoch 17/70
 - 3s - loss: 1.0514 - acc: 0.6698
Epoch 18/70
 - 3s - loss: 1.0397 - acc: 0.6733
Epoch 19/70
 - 3s - loss: 1.0323 - acc: 0.6738
Epoch 20/70
 - 3s - loss: 1.0243 - acc: 0.6766
Epoch 21/70
 - 3s - loss: 1.0161 - acc: 0.6759
Epoch 22/70
 - 3s - l

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 1214.0min finished
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=170)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/70
 - 13s - loss: 2.8372 - acc: 0.2619
Epoch 2/70
 - 4s - loss: 1.9519 - acc: 0.4744
Epoch 3/70
 - 4s - loss: 1.6059 - acc: 0.5604
Epoch 4/70
 - 4s - loss: 1.4463 - acc: 0.5954
Epoch 5/70
 - 4s - loss: 1.3477 - acc: 0.6146
Epoch 6/70
 - 4s - loss: 1.2819 - acc: 0.6290
Epoch 7/70
 - 4s - loss: 1.2358 - acc: 0.6379
Epoch 8/70
 - 4s - loss: 1.2022 - acc: 0.6430
Epoch 9/70
 - 4s - loss: 1.1731 - acc: 0.6476
Epoch 10/70
 - 4s - loss: 1.1531 - acc: 0.6520
Epoch 11/70
 - 4s - loss: 1.1363 - acc: 0.6537
Epoch 12/70
 - 4s - loss: 1.1224 - acc: 0.6566
Epoch 13/70
 - 4s - loss: 1.1107 - acc: 0.6579
Epoch 14/70
 - 4s - loss: 1.0971 - acc: 0.6605
Epoch 15/70
 - 4s - loss: 1.0839 - acc: 0.6631
Epoch 16/70
 - 4s - loss: 1.0721 - acc: 0.6646
Epoch 17/70
 - 4s - loss: 1.0673 - acc: 0.6661
Epoch 18/70
 - 4s - loss: 1.0524 - acc: 0.6701
Epoch 19/70
 - 4s - loss: 1.0490 - acc: 0.6710
Epoch 20/70
 - 4s - loss: 1.0406 - acc: 0.6722
Epoch 21/70
 - 4s - loss: 1.0298 - acc: 0.6738
Epoch 22/70
 - 5s - l

'20:19:03.712927'

In [21]:
pd.DataFrame(RS3.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_nodes_l3', 'param_nodes_l2', 'param_nodes_l1',
       'param_dropout_l3', 'param_dropout_l2', 'param_dropout_l1',
       'param_batch_size', 'params', 'split0_test_neg_log_loss',
       'split1_test_neg_log_loss', 'split2_test_neg_log_loss',
       'split3_test_neg_log_loss', 'mean_test_neg_log_loss',
       'std_test_neg_log_loss', 'rank_test_neg_log_loss',
       'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy',
       'split3_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy',
       'rank_test_accuracy'],
      dtype='object')

In [22]:
RS3.best_params_

{'nodes_l3': 50,
 'nodes_l2': 120,
 'nodes_l1': 170,
 'dropout_l3': 0.2,
 'dropout_l2': 0.30000000000000004,
 'dropout_l1': 0.30000000000000004,
 'batch_size': 3000}

In [23]:
RS3.best_score_

0.707234985471497

In [24]:
cols = ['rank_test_neg_log_loss', 'mean_test_neg_log_loss', 'rank_test_accuracy', 'mean_test_accuracy', 
        'param_nodes_l1', 'param_nodes_l2', 'param_nodes_l3', 'param_dropout_l1', 'param_dropout_l2', 'param_dropout_l3', 'param_batch_size']

In [25]:
df = pd.DataFrame(RS3.cv_results_)
df[cols]

,rank_test_neg_log_loss,mean_test_neg_log_loss,rank_test_accuracy,mean_test_accuracy,param_nodes_l1,param_nodes_l2,param_nodes_l3,param_dropout_l1,param_dropout_l2,param_dropout_l3,param_batch_size
0,8,-0.864630,28,0.702908,150,120,70,0.1,0.2,0.2,3000
1,19,-0.872966,21,0.705228,180,90,80,0.1,0.1,0.4,2000
2,29,-0.895727,30,0.701810,150,140,50,0.1,0.2,0.4,500
3,17,-0.868403,9,0.706482,190,120,70,0.3,0.4,0.1,500
4,15,-0.867779,7,0.706524,160,130,70,0.4,0.3,0.1,3000
5,3,-0.861143,4,0.706618,180,140,80,0.4,0.1,0.4,2000
6,6,-0.864322,8,0.706503,180,120,50,0.3,0.1,0.4,1000
7,24,-0.883255,12,0.706190,150,100,50,0.2,0.4,0.3,1000
8,9,-0.865108,3,0.706650,160,120,70,0.4,0.3,0.1,2000
9,18,-0.870023,10,0.706252,160,90,60,0.4,0.2,0.1,500


In [26]:
df.loc[df.rank_test_accuracy<16, cols]

,rank_test_neg_log_loss,mean_test_neg_log_loss,rank_test_accuracy,mean_test_accuracy,param_nodes_l1,param_nodes_l2,param_nodes_l3,param_dropout_l1,param_dropout_l2,param_dropout_l3,param_batch_size
3,17,-0.868403,9,0.706482,190,120,70,0.3,0.4,0.1,500
4,15,-0.867779,7,0.706524,160,130,70,0.4,0.3,0.1,3000
5,3,-0.861143,4,0.706618,180,140,80,0.4,0.1,0.4,2000
6,6,-0.864322,8,0.706503,180,120,50,0.3,0.1,0.4,1000
7,24,-0.883255,12,0.706190,150,100,50,0.2,0.4,0.3,1000
8,9,-0.865108,3,0.706650,160,120,70,0.4,0.3,0.1,2000
9,18,-0.870023,10,0.706252,160,90,60,0.4,0.2,0.1,500
12,5,-0.861874,4,0.706618,190,110,70,0.3,0.2,0.1,500
14,1,-0.860343,15,0.705824,150,140,80,0.4,0.3,0.1,1000
15,13,-0.867015,1,0.707235,170,120,50,0.3,0.3,0.2,3000
